# 基于 KITTI 数据集训练点云检测模型

## 整理数据

1. 下载 [kitti_tiny_3D](https://1drv.ms/u/s!AurVFREJAxzLd8bAPU29JMRE-v8?e=eiqw7r) 数据集，解压到 data/kitti 路径，该数据从 KITTI 数据集中裁剪而来
1. 参照 https://mmdetection3d.readthedocs.io/en/latest/datasets/kitti_det.html 准备数据文件

In [ ]:
!curl -sL -o kitti_tiny_3D.zip 'https://onedrive.live.com/download?resid=CB1C03091115D5EA%21119&authkey=!AO57a1ru2Tz2jHQ'
!unzip -d data/kitti kitti_tiny_3D.zip

In [1]:
!python tools/create_data.py kitti --root-path ./data/kitti --out-dir ./data/kitti --extra-tag kitti

Generate info. this may take several minutes.
[                                                  ] 0/25, elapsed: 0s, ETA:
[>                                 ] 1/25, 1.3 task/s, elapsed: 1s, ETA:    19s
[>>                                ] 2/25, 1.8 task/s, elapsed: 1s, ETA:    13s
[>>>>                              ] 3/25, 2.7 task/s, elapsed: 1s, ETA:     8s
[>>>>>                             ] 4/25, 3.6 task/s, elapsed: 1s, ETA:     6s
[>>>>>>                            ] 5/25, 4.4 task/s, elapsed: 1s, ETA:     5s
[>>>>>>>>                          ] 6/25, 5.3 task/s, elapsed: 1s, ETA:     4s
[>>>>>>>>>                         ] 7/25, 6.2 task/s, elapsed: 1s, ETA:     3s
[>>>>>>>>>>                        ] 8/25, 7.0 task/s, elapsed: 1s, ETA:     2s
[>>>>>>>>>>>>                      ] 9/25, 7.9 task/s, elapsed: 1s, ETA:     2s
[>>>>>>>>>>>>>                    ] 10/25, 8.8 task/s, elapsed: 1s, ETA:     2s
[>>>>>>>>>>>>>>                   ] 11/25, 9.5 task/s, elapsed: 1s, ETA:     

## 加载&查看默认配置文件

In [2]:
!mim download mmdet3d --config hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class --dest checkpoints

processing hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class...
hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class_20220301_150306-37dc2420.pth exists in C:\Users\WRH\Desktop\mmdetection3d\checkpoints
Successfully dumped hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class.py to C:\Users\WRH\Desktop\mmdetection3d\checkpoints


C:\Users\WRH\miniconda3\envs\mm38\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [3]:
from mmcv import Config

In [4]:
cfg = Config.fromfile('configs/pointpillars/hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class.py')

In [5]:
print(cfg.pretty_text)

voxel_size = [0.16, 0.16, 4]
model = dict(
    type='VoxelNet',
    voxel_layer=dict(
        max_num_points=32,
        point_cloud_range=[0, -39.68, -3, 69.12, 39.68, 1],
        voxel_size=[0.16, 0.16, 4],
        max_voxels=(16000, 40000)),
    voxel_encoder=dict(
        type='PillarFeatureNet',
        in_channels=4,
        feat_channels=[64],
        with_distance=False,
        voxel_size=[0.16, 0.16, 4],
        point_cloud_range=[0, -39.68, -3, 69.12, 39.68, 1]),
    middle_encoder=dict(
        type='PointPillarsScatter', in_channels=64, output_shape=[496, 432]),
    backbone=dict(
        type='SECOND',
        in_channels=64,
        layer_nums=[3, 5, 5],
        layer_strides=[2, 2, 2],
        out_channels=[64, 128, 256]),
    neck=dict(
        type='SECONDFPN',
        in_channels=[64, 128, 256],
        upsample_strides=[1, 2, 4],
        out_channels=[128, 128, 128]),
    bbox_head=dict(
        type='Anchor3DHead',
        num_classes=3,
        in_channels=384,
  

# 编写配置文件并训练

保存如下内容到 `myconfigs.py` 文件,由于我们使用的就是 KITTI 数据集，并且已经按照约定保存在 `data/kitti` 目录下，只需要简单修改训练轮数及相关数字即可，修改可通过配置文件的继承实现。

```python
_base_ = [
    'configs\pointpillars\hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class.py'
]

data = dict(
    samples_per_gpu=4,
    workers_per_gpu=1,
    persistent_workers=True,
    test=dict(
        split='testing',
        ann_file='data/kitti/kitti_infos_test.pkl',
    ))

runner = dict(max_epochs=5)
checkpoint_config = dict(interval=5)
evaluation = dict(interval=5)
log_config = dict(interval=10)

load_from = 'checkpoints/hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class_20220301_150306-37dc2420.pth'

```

使用命令行工具启动训练

In [6]:
!python .\tools\train.py myconfig.py

[                                                  ] 0/5, elapsed: 0s, ETA:
[>>>>>>                            ] 1/5, 0.1 task/s, elapsed: 12s, ETA:    47s
[>>>>>>>>>>>>>                     ] 2/5, 0.2 task/s, elapsed: 12s, ETA:    18s
[>>>>>>>>>>>>>>>>>>>>              ] 3/5, 0.3 task/s, elapsed: 12s, ETA:     8s
[>>>>>>>>>>>>>>>>>>>>>>>>>>>       ] 4/5, 0.3 task/s, elapsed: 12s, ETA:     3s
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 5/5, 0.4 task/s, elapsed: 12s, ETA:     0s
Converting prediction to KITTI format
[                                                  ] 0/5, elapsed: 0s, ETA:
[>>>>>>                           ] 1/5, 250.7 task/s, elapsed: 0s, ETA:     0s
[>>>>>>>>>>>>>                    ] 2/5, 334.2 task/s, elapsed: 0s, ETA:     0s
[>>>>>>>>>>>>>>>>>>>              ] 3/5, 376.0 task/s, elapsed: 0s, ETA:     0s
[>>>>>>>>>>>>>>>>>>>>>>>>>>       ] 4/5, 401.1 task/s, elapsed: 0s, ETA:     0s
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 5/5, 417.8 task/s, elapsed: 0s, ETA:     0s
Result is 

2022-11-03 19:22:45,595 - mmdet - INFO - Environment info:
------------------------------------------------------------
sys.platform: win32
Python: 3.8.13 (default, Mar 28 2022, 06:59:08) [MSC v.1916 64 bit (AMD64)]
CUDA available: True
GPU 0: NVIDIA GeForce GTX 1080
CUDA_HOME: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v10.2
NVCC: Cuda compilation tools, release 10.2, V10.2.8
MSVC: 用于 x64 的 Microsoft (R) C/C++ 优化编译器 19.29.30146 版
GCC: n/a
PyTorch: 1.8.2
PyTorch compiling details: PyTorch built with:
  - C++ Version: 199711
  - MSVC 192930040
  - Intel(R) Math Kernel Library Version 2020.0.2 Product Build 20200624 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v1.7.0 (Git Hash 7aed236906b1f7a05c0917e5257a1af05e9ff683)
  - OpenMP 2019
  - CPU capability usage: AVX2
  - CUDA Runtime 10.2
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_61,code=sm_61;-genco

训练好的模型保存在 `work_dirs/myconfig/latest.pth`

## 测试训练的模型

In [7]:
!python .\tools\test.py \
    myconfig.py \
    work_dirs/myconfig/latest.pth \
    --eval mAP
    # --show --show-dir tmp

load checkpoint from local path: work_dirs/myconfig/latest.pth
[                                                  ] 0/20, elapsed: 0s, ETA:
[>                                ] 1/20, 0.0 task/s, elapsed: 57s, ETA:  1075s
[>>>                              ] 2/20, 0.0 task/s, elapsed: 61s, ETA:   551s
[>>>>                             ] 3/20, 0.0 task/s, elapsed: 62s, ETA:   352s[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]

[>>>>>>                           ] 4/20, 0.1 task/s, elapsed: 63s, ETA:   251s
[>>>>>>>>                         ] 5/20, 0.1 task/s, elapsed: 63s, ETA:   190s[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]

[>>>>>>>>>                        ] 6/20, 0.1 task/s, elapsed: 64s, ETA:   150s
[>>>>>>>>>>>                      ] 7/20, 0.1 task/s, elapsed: 65s, ETA:   120s
[>>>>>>>>>>>>>                    ] 8/20, 0.1 task/s, elapsed: 65s, ETA:    98s
[>>>>>>>>>>>>>>                   ] 9/20, 0.1 task/s, elapsed: 66s, ETA:    

c:\users\wrh\desktop\mmdetection3d\mmdet3d\models\dense_heads\anchor3d_head.py:84: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  warnings.warn(
